In [1]:
%matplotlib qt


import numpy as np
import torch
import torch.nn as nn
from auto_LiRPA import BoundedModule, BoundedTensor, PerturbationLpNorm
import torch.nn.functional as F

#constants
TARGET = np.array([[-0.1, 0.2, 0.3]], dtype = np.double)


EPS = 0.5

W0 = np.array([[-0.5, -0.03, -0.08],
               [ 0.15,  0.19,  0.27]], dtype = np.double)
W0 = np.random.uniform(-1, 1, (2,3))

B0 = np.array([-0.46, -0.02], dtype = np.double)
B0 = np.random.uniform(-1, 1, (2,))


W1 = np.array([[ 0.6880, -0.4974],
               [ -0.3441, 0.6595],
               [ 0.1442, -0.0750]], dtype = np.double)
W1 = np.random.uniform(-1, 1, (3,2))

B1 = np.array([ 0.5848, 0.2861, -0.0015], dtype = np.double)
B1 = np.random.uniform(-1, 1, (3,))


In [2]:
#a very simple neural net
class LinearModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc0 = nn.Linear(3, 2)
        self.fc0.weight = torch.nn.Parameter(torch.from_numpy(W0))
        self.fc0.bias = torch.nn.Parameter(torch.from_numpy(B0))
        
        self.fc1 = nn.Linear(2,3)
        self.fc1.weight = torch.nn.Parameter(torch.from_numpy(W1))
        self.fc1.bias = torch.nn.Parameter(torch.from_numpy(B1))
        
    def forward(self, x):
        return self.fc1(F.relu(self.fc0(x)))
        
#Compute bound using Interval Bound Propagation, using auto_LiRPA API
target = torch.from_numpy(TARGET)
                        
ball = PerturbationLpNorm(norm=np.inf, eps=EPS)
ball_tensor = BoundedTensor(target, ball)
print(ball_tensor)

original_model = LinearModel()
print(original_model.fc1.weight, original_model.fc1.bias)
lirpa_model = BoundedModule(original_model, torch.empty_like(target))

print(lirpa_model(ball_tensor))

# lb, ub = lirpa_model.compute_bounds(IBP=True, method = 'forward')
lb, ub = lirpa_model.compute_bounds(x = (ball_tensor, ), IBP=True, method = 'forward')
ibp_bound = np.array([lb.detach().numpy(), ub.detach().numpy()])
print(ibp_bound)

<BoundedTensor: tensor([[-0.1000,  0.2000,  0.3000]], dtype=torch.float64), PerturbationLpNorm(norm=inf, eps=0.5)>
Parameter containing:
tensor([[-0.7751,  0.0131],
        [ 0.5026, -0.0678],
        [-0.5925,  0.9591]], dtype=torch.float64, requires_grad=True) Parameter containing:
tensor([0.1461, 0.6308, 0.4676], dtype=torch.float64, requires_grad=True)
tensor([[0.1463, 0.6298, 0.4815]], dtype=torch.float64, grad_fn=<AddBackward0>)
[[[-0.05304209  0.57648807  0.31540735]]

 [[ 0.15660273  0.7599076   1.23556742]]]


/home/nle/anaconda3/envs/py38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
#Compute the output of the network
def forward(x):
    fc0 = np.matmul(x, W0.transpose()) + B0
    fc1 = np.matmul(np.maximum(fc0, 0), W1.transpose()) + B1
    return fc1
print(lirpa_model(ball_tensor).detach().numpy())
print(forward(TARGET))
assert(np.array_equal(forward(TARGET),
                      lirpa_model(ball_tensor).detach().numpy()))

[[0.14627333 0.62981573 0.4814717 ]]
[[0.14627333 0.62981573 0.4814717 ]]


In [4]:
##Compute bound using Interval Bound Propagation, using my own implementation.
##The closed-form solution is in eq(6), https://arxiv.org/pdf/1810.12715.pdf
my_ball = np.array([TARGET - EPS, TARGET + EPS])
print(my_ball.shape)

def my_IBP(prev_bound: np.array, W: np.array, b: np.array):
    """
    prev_bound: 2x784 prev_bound[0][i]: lower of unit ith, prev_bound[1]i]: upper of unit ith
    W: 784x256
    b: 256
    """
    assert(prev_bound.shape[0]==2)
    assert(prev_bound.shape[-1]==W.shape[0])
    assert(W.shape[1] == b.shape[0])
    
    prev_u = (prev_bound[0,:] + prev_bound[1,:])/2

    prev_r = (prev_bound[1,:] - prev_bound[0,:])/2
    
    u = np.matmul(prev_u , W) + b
    
    r = np.matmul(prev_r , abs(W) )
    
    new_bound_lower = u - r
    
    new_bound_upper = u + r
    
    new_bound = np.array([new_bound_lower, new_bound_upper])
    
    return new_bound


fc0_bound = np.maximum(my_IBP(my_ball, W0.transpose(), B0), 0)
fc1_bound = my_IBP(fc0_bound, W1.transpose(), B1)


print("my_bound:\n", fc1_bound)
print("ibp_bound:\n", ibp_bound)

#cannot use array_equal due to some floating point difference?
assert(np.allclose(fc1_bound, 
                   ibp_bound))


(2, 1, 3)
my_bound:
 [[[-0.05304209  0.57648807  0.31540735]]

 [[ 0.15660273  0.7599076   1.23556742]]]
ibp_bound:
 [[[-0.05304209  0.57648807  0.31540735]]

 [[ 0.15660273  0.7599076   1.23556742]]]


In [5]:
## Computer bound using CROWN
print(lirpa_model(ball_tensor))
crown_lb, crown_ub = lirpa_model.compute_bounds(x = (ball_tensor, ), method = 'backward')
crown_bound = np.array([crown_lb.detach().numpy(), crown_ub.detach().numpy()])
print("crown_bound:\n", crown_bound)


tensor([[0.1463, 0.6298, 0.4815]], dtype=torch.float64, grad_fn=<AddBackward0>)
crown_bound:
 [[[-0.05827196  0.57648807 -0.35947725]]

 [[ 0.15660273  0.78690792  1.23556742]]]


In [8]:
#sampling
N_POINTS = 10**3

samples = []
for i in range(N_POINTS):
    x = np.random.uniform(my_ball[0][0][0], my_ball[1][0][0])
    y = np.random.uniform(my_ball[0][0][1], my_ball[1][0][1])
    z = np.random.uniform(my_ball[0][0][2], my_ball[1][0][2])
    samples.append(np.array([[x,y,z]], dtype = np.double))

s0 = samples[0]
    
samples = np.concatenate(samples, axis = 0)
print(samples.shape)
samples = torch.from_numpy(samples)
output = lirpa_model(samples).detach().numpy()
# output = np.maximum(np.matmul(samples, W0.transpose()) + B0, 0)
print(output)
print(output.shape)

import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure()
ax = plt.axes(projection='3d')

ax.scatter3D(output[:,0], output[:,1], output[:, 2])
# ax.scatter3D(samples[:,0], samples[:,1], samples[:,2], c = 'red')
print(output[0])
print(forward(s0))

(1000, 3)
[[0.15390594 0.59041081 1.03868873]
 [0.14781628 0.62184996 0.59411402]
 [0.14608355 0.63079549 0.46761704]
 ...
 [0.15231222 0.5986387  0.92233983]
 [0.1517188  0.60170234 0.87901746]
 [0.14634387 0.62945155 0.4866215 ]]
(1000, 3)
[0.15390594 0.59041081 1.03868873]
[[0.15390594 0.59041081 1.03868873]]


In [17]:
#my CROWN
layers = [{'name': 'fc0', 'W': W0, 'B': B0},
          {'name': 'fc1', 'W': W1, 'B': B1}
         ]

def my_crown(layers, ball):
    cur_bound = ball
    for idx, l in enumerate(layers):
        if idx == 0:
            cur_bound= my_IBP(cur_bound, l['W'].transpose(), l['B'])
        else:
            cur_bound = my_IBP(np.maximum(cur_bound, 0), l['W'].transpose(), l['B'])
                                       
        l['ibp_bound'] = cur_bound
        
    for l in layers:
        print(l['name'])
        n_relus = l['ibp_bound'].shape[-1]
        
        for idx in range(n_relus):
            lower = l['ibp_bound'][0][0][idx]
            upper = l['ibp_bound'][1][0][idx]
            assert(lower < upper)
            print(lower, upper)
            if upper < 0:
                print("always neg")
            elif lower >= 0:
                print("always pos")
            else:
                print("need bound")
        
my_crown(layers, my_ball)

fc0
-1.4867349782184158 0.25689241410857033
need bound
-0.7718139458004731 0.8007051273689274
need bound
fc1
-0.053042088665785914 0.1566027267833884
need bound
0.5764880715605143 0.759907597312715
always pos
0.3154073534893366 1.2355674246152804
always pos
